<h1 style="text-align: center">
<strong>WEB SCRAPING - REPOSITORIO DE UNSA</strong>
</h1>

**Nombre:** Oliverio Pichardo Diestra

**Descripción del caso:**
Se quiere obtener información estructurada de las tesis de pregrado y posgrado de Estadística 
Informática o de carreras similares desarrolladas a nivel  nacional, a través de los repositorios 
institucionales de las universidades.

**Especificaciones:**
1. Debido a que se menciona carreras similares y no se especifica terminología alguna para ello, 
entonces se procede a considerar lo establecido por INEI en su *Clasificador Nacional de Programas e Instituciones de 
Educación Superior Universitaria, Pedagógica, Tecnológica y Técnico Productiva, 2018*. (Ver Tabla 1)

    **Tabla 1**  
    *Programas similares a Estadística a nivel de profesional (pregrado)*
    | CÓDIGO |                      PROGRAMA                     |
    |:------:|:-------------------------------------------------:|
    | 542016 |                    Estadística                    |
    | 542026 |             Estadística e informática             |
    | 542036 |              Estadística informática              |
    | 542046 | Estadística para la gestión de servicios de salud |
    | 542056 |               Ingeniería estadística              |
    | 542066 |        Ingeniería estadística e informática       |
    | 542076 |         Ingeniería estadística informática        |
    | 542996 |           Otros programas en estadística          |

    *Nota: Adaptado del "Clasificador Nacional de Programas e Instituciones de 
    Educación Superior Universitaria, Pedagógica, Tecnológica y Técnico Productiva, 2018" por INEI.*

2. El repositorio escogido es el de la Universidad Nacional de San Agustín de Arequipa (UNSA). 
El respectivo url de dicho repositorio es:  http://repositorio.unsa.edu.pe/browse?type=dateissued **(url-principal)**


# 1. Web Scraping

## 1.1. Librerías, módulos o paquetes
A fin de realizar la extracción de datos pertinentes, se emplearán las siguientes librerías, módulos o paquetes:

In [1]:
# Cargando librerías, módulos o paquetes pertinentes
# %pip install requests
# %pip install pandas
import requests
from bs4 import BeautifulSoup
import re
import itertools
import pandas as pd

## 1.2. Funciones
Se definirán dos funciones las cuales tendrán los siguientes objetivos respectivamente:
1. Extraer todas las urls de la diversas páginas de la *url-principal* en las que se encuentran las tesis de pregrado y posgrado. Tales urls serán denominadas *url-página*.
2. De cada *url-página*, extraer las urls que contienen la información específica de cada una las tesis. A estas les denominaremos *url-publicación*.

A fin de comprender mejor lo mencionado, veamos la estructura jerárquica:
1. Nivel 1 (url-principal): http://repositorio.unsa.edu.pe/browse?type=dateissued
2. Nivel 2 (url-página): Es una subpágina de la url-principal que contiene en promedio 20 publicaciones. Hasta la fecha, en total, existen 682 subpáginas.
3. Nivel 3 (url-publicación): Es una subpágina de una url-página que contiene la información específica de una determinada publicación. Hasta el momento, contiene en promedio 28 descriptores de la respectiva publicación.

### 1.2.1. Extracción de todas las url-página de la url-principal
A fin de realizar la extracción de todas las url-página de la url-principal, se identificó un patrón de url que sirve para generar todas las url-página. A esa se le denominará *url-base*.
A continuación, se define una función en la que:
1. se identifica la url-base,
2. se crea una lista de los caracteres que se deben reemplazar en el url-base para generar todas la url-página y
3. se generan las url-página mediante la sustitución de unos caracteres específicos de la url-base por los elementos la lista generada en el punto 2.

In [2]:
# Creando una función que permite extraer todas las url-página
def all_pages(url):
    # Descargar el contenido de la url-principal
    page = requests.get(url)
    # Crear un objeto BeautifulSoup a partir del contenido de la página
    soup = BeautifulSoup(page.content, "html.parser")
    # Encontrar el tag a que contiene la url-base de las url-página
    pages = soup.find_all("a", {"class":"next-page-link"})
    # Extraer el url-base de las url-página
    pages_soup = BeautifulSoup(str(pages), 'html.parser')
    a_tags = pages_soup.find_all('a') # find all anchor elements
    base_href = ["http://repositorio.unsa.edu.pe/" + a_tag['href'] for a_tag in a_tags][0]
    # Crear lista de límites inferiores de cada url-página
    string_pages = soup.find_all("p", {"class":"pagination-info"})
    matches = re.findall("\d+", str(string_pages[0]))
    lower_limit_max = (max([int(item) for item in matches]) // 20) * 20
    lower_limits = list(range(0, lower_limit_max + 20, 20))
    # Crear lista de las url-página de la url-principal
    hrefs_all = [re.sub(r"offset=\d+", "offset=" + str(pag), base_href) for pag in lower_limits]
    return hrefs_all

### 1.2.2. Extracción de todas las url-publicación de una url-página
A fin de realizar la extracción de todas las url-publicación de una url-página, se identificó el tag que contiene la expresión extra añadir en http://repositorio.unsa.edu.pe que permite acceder hasta la información de una determina publicación.

In [3]:
# Creando una función que permite extraer todas las url-publicación
def links_scrapy(url):
    # Descargar el contenido de la url-página
    page = requests.get(url)
    # Crear un objeto BeautifulSoup a partir del contenido de la página
    soup = BeautifulSoup(page.content, "html.parser")
    # Encontrar el tag padre de las publicaciones que contiene su respectivo url-publicación
    publication = soup.find_all("div", {"class":"artifact-title"})
    # Extraer los url-publicación de los tags hijos respectivos
    publication_soup = BeautifulSoup(str(publication), 'html.parser')
    a_tags = publication_soup.find_all('a')
    hrefs = ["http://repositorio.unsa.edu.pe" + a_tag['href'] + "?show=full" for a_tag in a_tags]
    return hrefs

## 1.3. Aplicación: Extracción de las url-publicación del repositorio
A continuación, se obtendrá todas las url-publicación del repositorio.

In [ ]:
# NOTA: no se recomienda ejecutar esta celda, ya que toma algo de tiempo obtener un output (9 min aprox.)
# Extraer todas las url-página de la url-principal
## every_page: es una lista que contiene las url-página
every_page = all_pages("http://repositorio.unsa.edu.pe/browse?type=dateissued")
# Extraer todas las url-publicación de cada una de las url-pagina
## publications_links: es un a lista que contiene todas las url-publicación
## publications_links se obtuvo de aplanar (flatten) la lista de listas de url-publicación de cada una de las url-página
publications_links = list(itertools.chain(*[links_scrapy(url) for url in every_page]))

Veamos la ejecución de lo anterior para los dos primeras url-página.

In [4]:
# EJEMPLO DE EJECUCIÓN
# En este ejemplo solo se considera las dos primeras páginas del repositorio
every_page = all_pages("http://repositorio.unsa.edu.pe/browse?type=dateissued")
publications_links = list(itertools.chain(*[links_scrapy(url) for url in every_page[0:2]]))

## 1.4. Extracción de la información solicitada
Se procederá a extraer la información de cada una de las url-publicación.

In [5]:
# Encabezado para la extracción de la tabla
headers = {"User-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
# Crear las listas donde se almacenará la información de las tesis
anios, instituciones, titulos, autores, contribuidores, grados, resumenes = [], [], [], [], [], [], []
# Añadir un contador que será de utilidad para colocar un dato vacío en el caso de que
# la tesis no cuente con la información que se requiera
n = 0

In [8]:
for i in range(len(publications_links)):
    respuesta = requests.get(publications_links[i], headers=headers)
    tabla_tesis = pd.read_html(respuesta.content, encoding = 'utf8')
    tabla_1 = tabla_tesis[0]
    # Seleccionar el dato de la fecha de tesis de cada link y añadir en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.date.issued':
            año=tabla_1.iloc[j,1]
            anios.append(año)
            n = n + 1
    if n == 0:
        anios.append(" ")
    else:
        n = 0
    #Seleccionar el dato de la institución de cada link y añadir en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.publisher':
            institucion=tabla_1.iloc[j,1]
            instituciones.append(institucion)
            n = n + 1
    if n == 0:
        instituciones.append(" ")
    else:
        n = 0
    # seleccionar el dato del titulo de cada link y añadir en su respectiva lista
    for j in range(len(tabla_1)):        
        if tabla_1.iloc[j,0] == 'dc.title':
            titulo=tabla_1.iloc[j,1]
            titulos.append(titulo)
            n = n + 1
    if n == 0:
            titulos.append(" ")
    else:
        n = 0
    #seleccionar el dato del autor de tesis de cada link y añadir en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.contributor.author':
            autor=tabla_1.iloc[j,1]
            autores.append(autor)
            n = n + 1
    # En caso exista más de un autor, sólo seleccionar el primero
            break
    if n == 0:
            autores.append(" ")
    else:
        n = 0 
    #seleccionar el dato del grado de tesis de cada link y añadir en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'thesis.degree.name':
            grado=tabla_1.iloc[j,1]
            grados.append(grado)
            n = n + 1
    if n == 0:
            grados.append(" ")
    else:
        n = 0
    #seleccionar el dato del asesor de tesis de cada link y añadir en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.contributor.advisor':
            contribuidor=tabla_1.iloc[j,1]
            contribuidores.append(contribuidor)
            n = n + 1
    #En caso exista más de un asesor, sólo seleccionar el primero
            break
    if n == 0:
            contribuidores.append(" ")
    else:
        n = 0
    #seleccionar el dato del resumen de tesis de cada link y añadir en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.description.abstract':
            resumen=tabla_1.iloc[j,1]
            resumenes.append(resumen)
            n = n + 1
    if n == 0:
            resumenes.append(" ")
    else:
        n = 0

## 1.5. Almacenamiento de la información solicitada
Se procederá a almacenar la información de cada una de las url-publicación en un dataframe.

In [10]:
# Crear un data frame donde se almacenará todos los datos de las publicaciones
df = pd.DataFrame()

# Asignar cada lista a una determinada columna del dataframe
df['instituciones'] = instituciones
df['titulos'] = titulos
df['autores'] = autores
df['contribuidores'] = contribuidores
df['resumenes'] = resumenes
df['grados'] = grados
df['anios'] = anios

## 1.6. Exportación de la información solicitada
Se procederá a exportar la información solicitada en un archivo .csv